In [ ]:
using Plot

using GSEA

In [ ]:
use_style!()

;

In [ ]:
element_, element_score_, set_element_ = make_benchmark("card")

;

In [ ]:
using Printf: @sprintf

using Plotly: Layout, attr, plot, scatter
using StatsBase: mean

function plot_scoring_set(
    element_::Vector{String},
    element_score_::Vector{Float64},
    set_element_::Vector{String};
    width::Real = 800,
    height::Real = 500,
    line_width::Real = 2,
    title1_text::String = "Set Enrichment",
    title2_text::String = "",
    title1_font_size::Int64 = 20,
    element_value_name::String = "Element<br>Value",
    axis_title_font_size::Int64 = 12,
)::Any

    n_element = length(element_)

    annotation_template = attr(
        xref = "paper",
        yref = "paper",
        xanchor = "center",
        yanchor = "middle",
        showarrow = false,
    )

    x_annotation_template = merge(annotation_template, attr(x = 0.5))

    y_annotation_template = merge(annotation_template, attr(x = -0.125, font_size = axis_title_font_size))

    title2_font_size = title1_font_size * 0.7

    yaxis1_domain = (0, 0.3)

    yaxis2_domain = (0.3, 0.4)

    yaxis3_domain = (0.4, 1)

    layout = Layout(
        width = width,
        height = height,
        margin_l = width * 0.2,
        margin_t = height * 0.2,
        legend_orientation = "h",
        legend_x = 0.5,
        legend_y = -0.2,
        legend_xanchor = "center",
        legend_yanchor = "middle",
        xaxis1_zeroline = false,
        xaxis1_showspikes = true,
        xaxis1_spikemode = "across",
        xaxis1_spikedash = "solid",
        yaxis3_domain = yaxis3_domain,
        yaxis3_showline = true,
        yaxis2_domain = yaxis2_domain,
        yaxis2_showticklabels = false,
        yaxis2_showgrid = false,
        yaxis1_domain = yaxis1_domain,
        yaxis1_showline = true,
        annotations = [
            merge(
                x_annotation_template,
                attr(y = 1.25, text = "<b>$title1_text</b>", font_size = title1_font_size),
            ),
            merge(
                x_annotation_template,
                attr(y = 1.15, text = "<b>$title2_text</b>", font_size = title2_font_size),
            ),
            merge(
                x_annotation_template,
                attr(
                    y = -0.1,
                    text = "<b>Element Rank (n=$n_element)</b>",
                    font_size = axis_title_font_size,
                ),
            ),
            merge(
                y_annotation_template,
                attr(y = mean(yaxis3_domain), text = "<b>Set<br>Enrichment</b>"),
            ),
            merge(
                y_annotation_template,
                attr(y = mean(yaxis2_domain), text = "<b>Set<br>Member</b>"),
            ),
            merge(
                y_annotation_template,
                attr(y = mean(yaxis1_domain), text = "<b>$element_value_name</b>"),
            ),
        ],
    )

    x = 1:n_element

    element_score_, element_ = sort_like((element_score_, element_))

    element_score_trace = scatter(
        name = "Element Value",
        x = x,
        y = element_score_,
        text = element_,
        line_width = line_width,
        line_color = "#ffb61e",
        fill = "tozeroy",
    )

    is_ = check_is(element_, set_element_)

    set_element_bit = BitVector(is_)

    set_element_trace = scatter(
        name = "Set Element",
        yaxis = "y2",
        x = x[set_element_bit],
        y = zeros(Int64(sum(is_))),
        text = element_[set_element_bit],
        mode = "markers",
        marker_symbol = "line-ns-open",
        marker_size = height * (yaxis2_domain[2] - yaxis2_domain[1]) * 0.25,
        marker_line_width = line_width,
        marker_color = "#006c7f",
        hoverinfo = "name+x+text",
    )

    set_score_, extreme, area =
        score_set_pk(element_, element_score_, set_element_)

    p_value = 0.008

    extreme_string = @sprintf "%.2e" extreme

    area_string = @sprintf "%.2e" area

    p_value_string = @sprintf "%.2e" p_value

    push!(
        layout["annotations"],
        merge(
            x_annotation_template,
            attr(
                y = 1.05,
                text = join(
                    ("Extreme = $extreme_string", "Area = $area_string", "P-Value = $p_value_string"),
                    "     ",
                ),
                font_size = title2_font_size,
                font_color = "#913228",
            ),
        ),
    )

    set_score_trace = scatter(
        name = "Set Enrichment",
        yaxis = "y3",
        x = x,
        y = set_score_,
        text = element_,
        line_width = line_width,
        line_color = "#8db255",
        fill = "tozeroy",
    )

    return plot([element_score_trace, set_element_trace, set_score_trace], layout)

end

plot_scoring_set(element_, element_score_, set_element_)